In [127]:
import numpy as np

#Natural Constants
e = 1.602176e-19 #Coulombs (SI)
m = 9.109384e-31 #kg (SI)
c = 2.9979e8 #m/s (SI)
π = np.pi
L = 4.925*2.54/100 #m (SI)

#Measured B (from spreadsheet)
Bdz = 7.7336e-3 #T-m
λr,λg,λb,λv,λh = 620e-9,532e-9,473e-9,405e-9,587.56e-9 #meters

λs = {'λr':λr,'λg':λg,'λb':λb,'λv':λv,'λh':λh}
ωr,ωg,ωb,ωv,ωh = c/λr,c/λg,c/λb,c/λv,c/λh
ωs = {'ωr':ωr,'ωg':ωg,'ωb':ωb,'ωv':ωv,'ωh':ωh}

class Sample:
    def __init__(self,name,Bs=[],Cs=[],nd=None,Vd=None):
        self.name = name
        self.Bs = Bs
        self.B1 = float(Bs[0])
        self.B2 = float(Bs[1])
        self.B3 = float(Bs[2])
        self.Cs = Cs
        self.C1 = float(Cs[0])
        self.C2 = float(Cs[1])
        self.C3 = float(Cs[2])
        self.nd = nd
        self.Vd = Vd
        
    def n_lamda(self,lam_meters):
        #This function must use lambda in microns to produce n!
        lam = lam_meters*1e6
        nsq = 1+(self.B1*lam**2)/(lam**2-self.C1)+(self.B2*lam**2)/(lam**2-self.C2)+(self.B3*lam**2)/(lam**2-self.C3)
        return np.sqrt(nsq)
    
    def n_omega(self,omega):
        lam = c/omega
        return self.n_lamda(lam)
    
    def dn_dλ(self,λ,dx=1e-4):
        dx=np.abs(λ*dx) # ← dx needs to be positive!
        dn_dλ = (self.n_lamda(λ+dx)-self.n_lamda(λ-dx))/(2*dx)
        return dn_dλ
    
    def dn_dω(self,ω,dx=1e-4):
        dx=np.abs(ω*dx) # ← dx needs to be positive!
        dn_dω = (self.n_omega(ω+dx)-self.n_omega(ω-dx))/(2*dx)
        return dn_dω
    
    def Verdet(self,λ):
        ω = c/λ
        print('\t λ = '+str(λ)+' m')
        print('\t ω = '+str(ω)+' /s')
        print('\t λ*self.dn_dλ(λ)= '+str(λ*self.dn_dλ(λ))+' [unitless]')
        print('\t ω*self.dn_dω(ω)= '+str(ω*self.dn_dω(ω))+' [unitless]')
        print('\t Coeffs e/2mc ='+str((e/(2*m*c)))+' /(T-m)')
        Vdet = (e/(2*m*c))*ω*self.dn_dω(ω)
        return Vdet
    
    def b(self):
        return b
    
    def a(self):
        return a
        
BK_7 = Sample(name='Schott BK-7',
              Bs=[1.03961212,2.31792344e-1,1.01046945],
              Cs=[6.00069867e-3,2.00179144e-2,1.03560653e2],
              nd=1.51680,
              Vd=64.17)

SF_57 = Sample(name='Schott SF-57',
              Bs=[1.87543831,3.7375749e-1,2.30001797],
              Cs=[1.41749518e-2,6.40509927e-2,1.77389795e2],
              nd=1.84666,
              Vd=23.78)

samples = [BK_7,SF_57]

for sample in samples:
    for key,value in λs.items():
        print(sample.name+' n('+key+'):')
        λ = float(value)
        ω = c/λ
        dndλ = sample.dn_dλ(λ)
        dndω = sample.dn_dω(ω)
        nλ = sample.n_lamda(λ)
        nω = sample.n_omega(ω)
        Vdet = sample.Verdet(λ) # rads/(T-m) [See dimensional analysis in notebook!]
        θ = Vdet*Bdz # Must multiply by Teslas! See dimensional analysis in notebook!
        print('\t nλ = '+str(nλ)+' m')
        print('\t nω = '+str(nω)+' /s')
        print('\t dndλ = '+str(dndλ)+' /m')
        print('\t dndω = '+str(dndω)+' s')
        print('\t \t Verdet = '+str(Vdet)+' rads/(T-m)')
        print('\t \t θ = '+str(θ)+' rad = '+str(360*θ/(2*π))+' degrees')

Schott BK-7 n(λr):
	 λ = 6.2e-07 m
	 ω = 483532258064516.1 /s
	 λ*self.dn_dλ(λ)= -0.022407772060084593 [unitless]
	 ω*self.dn_dω(ω)= 0.022407771688159883 [unitless]
	 Coeffs e/2mc =293.3418821763307 /(T-m)
	 nλ = 1.515539496351094 m
	 nω = 1.515539496351094 /s
	 dndλ = -36141.56783884612 /m
	 dndω = 4.634183410607134e-17 s
	 	 Verdet = 6.573137922382315 rads/(T-m)
	 	 θ = 0.05083401943653588 rad = 2.9125747693995008 degrees
Schott BK-7 n(λg):
	 λ = 5.32e-07 m
	 ω = 563515037593984.9 /s
	 λ*self.dn_dλ(λ)= -0.029463419080055786 [unitless]
	 ω*self.dn_dω(ω)= 0.029463418518282936 [unitless]
	 Coeffs e/2mc =293.3418821763307 /(T-m)
	 nλ = 1.5194725830654814 m
	 nω = 1.5194725830654814 /s
	 dndλ = -55382.36669183418 /m
	 dndω = 5.228506171562268e-17 s
	 	 Verdet = 8.642854643502073 rads/(T-m)
	 	 θ = 0.06684038067098763 rad = 3.829671713495397 degrees
Schott BK-7 n(λb):
	 λ = 4.73e-07 m
	 ω = 633805496828752.6 /s
	 λ*self.dn_dλ(λ)= -0.0371873243221188 [unitless]
	 ω*self.dn_dω(ω)= 0.03718732